# Neural Networks

In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
from keras.datasets import cifar10
import keras.utils.np_utils as np_utils
import numpy as np

## Load dataset

In [ ]:
def loadMNISTdataset():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    width, height = (x_train.shape[1], x_train.shape[2])

    # Reshape dataset to have a single channel, 
    # converting from integers to floats, and  
    # normalizing they range to [0-1]
    x_train = x_train.reshape(x_train.shape[0], 3*width*height).astype('float32') / 255
    x_test = x_test.reshape(x_test.shape[0], 3*width*height).astype('float32') / 255

    # Converting to one-hot encode for the target values
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    return (x_train, y_train, x_test, y_test, width, height)


In [ ]:
TODO

## Defining the model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

TODO

## Train Model

In [ ]:
nEpochs = 30
batchSize = 64

In [ ]:
TODO

In [ ]:
def plotTrainingHistory(history):
    plt.subplot(2, 1, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='gray', label='test')

    plt.subplot(2, 1, 2)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='orange', label='train')
    plt.plot(history.history['val_accuracy'], color='gray', label='test')

    plt.tight_layout()
    plt.show()

In [ ]:
TODO

## Test the model

In [ ]:
def showErrors(model, x_test, y_test, width, height):
    y_hat = np.argmax(model.predict(x_test), axis=-1)
    y_test_array = y_test.argmax(1)
    test_wrong = [im for im in zip(x_test,y_hat,y_test_array) if im[1] != im[2]]
    plt.figure(figsize=(15, 15))
    for ind, val in enumerate(test_wrong[:20]):
        plt.subplot(10, 10, ind + 1)
        im = 1 - val[0].reshape((width,height,3))
        plt.axis("off")
        plt.text(0, 0, val[2], fontsize=14, color='green') # correct
        plt.text(8, 0, val[1], fontsize=14, color='red')  # predicted
        plt.imshow(im, cmap='gray')
    plt.show()

In [ ]:
TODO